**Background**

I am looking to control a simple ON/OFF relay from Ultra96 using a GPIO and despite all of the really awesome Jupyter Notebooks out there showing custom hardware accelleration on interfacing with Programmable Logic using PYNQ, I haven't found a great example of controlling PS MIO GPIOs from a Jupyter Notebook.  After bugging a few folks about this, I came up with the following page which helps explain the thought process behind how it works and hopfully it serves as a good resource for others who are looking to do something similar. 

**It is just a single GPIO pin, how hard can it be?**

Well since the GPIO pin is controlled by Linux, we have to follow the Linux way of controlling GPIOs which is not exactly intuitive.

**Where are the GPIO controls and how can they be accessed?**

One of the most popular ways to control GPIOs from an embedded Linux platform is from the **sysfs** interface under /sys/class/gpio/ directory.  In fact you can see which GPIO controller devices are supported on your PYNQ system by listing the directory contents of that folder:

In [ ]:
!ls /sys/class/gpio/

Here the GPIO controller for PS MIO is enumerated as gpiochip338 which represents the base of the GPIOs that could potentially be connected to the Multiplexed Input Output (or MIO) controller.  Since 338 represents the beginning of the sequential GPIOs,  GPIO[0] would appear at the node /sys/class/gpio/gpio338 if we were to export it for control.  Since a number of MIO GPIOs areavailable and the logical numbers of those GPIOs are sequential, any higher MIO number would result in an equal offset from the base of 338.

<img src="images/mio_multiplexed_input_output.png">

Since we are targeting Ultra96 and I want to tap into the expansion header on Ultra96 to keep things as clean as possible, let us first look at the Ultra96-V2 schematic and see if there are any PS MIO pins on the low-speed expansion header.

<img src="images/ultra96_ps_mio45_gpio.png">

Here MIO[45] is on the outer row of the expansion header which makes it easy to get to from the edge of the board with an oscilloscope probe so I will probe that pin and use it for the example throughout the rest of this Notebook.

<img src="images/ultra96_probe_connections.png">

To probe that pin, I connected one half of one these 2mm row headers to my Ultra96:

<img src="images/ultra96_expansion_headers.png">

Amazon.com product listing:  [1x40P 2mm Straight Single Row Breakaway Male Pin Header](https://www.amazon.com/gp/product/B01GKJZ310/)

The target GPIO to be controlled first needs to be exported to sysfs control so that we can read and write to the device direction and value properties.  Since we are targeting the Zynq UltraScale+ MPSoC PS MIO[45] GPIO which is on the 40-pin low-speed expansion header of Ultra96, the translated Linux GPIO number will be 338 + 45 = 383 as illustrated in the following diagram:

<img src="images/ultra96_mio_mapping_to_linux_gpio.png">

The desired MIO GPIO control is exported to sysfs using the following command: 

In [ ]:
! echo 383 > /sys/class/gpio/export

**Okay, the GPIO is now exported for control.  How can the output value be changed?**

First, set the directionality of the exported GPIO control to "out" which sets the input/output mode to output mode. If you are watching the output on the oscillocope you will notice that the GPIO line immediately transitions from HIGH to LOW. This is due to the fact that the GPIO value is already set to logic LOW (with value set to 0) but that output does not actively get driven until the direction of the GPIO channel is set to output mode (by default the controller initialzes the mode to input).

In [ ]:
! echo out > /sys/class/gpio/gpio383/direction

Set the output value of the GPIO to 1 which sets the output logic level to HIGH.

After running the following command, observe the output value of MIO45 GPIO line will transition from 0V logic low to 1.8V logic high.

<img src="images/ultra96_gpio_scope_trigger_low_to_high.png">

In [ ]:
! echo 1 > /sys/class/gpio/gpio383/value

Next, set the output value of the GPIO back to 0 which sets the output logic level to LOW.

After running the following command, observe the output value of MIO45 GPIO line will transition from 1.8V logic high to 0V logic low.

<img src="images/ultra96_gpio_scope_trigger_high_to_low.png">

In [ ]:
! echo 0 > /sys/class/gpio/gpio383/value

**Conclusion**

A single GPIO can be used to control external circuitry via a MOSFET driver or relay driver.  These higher voltage/current controls can, in turn, be used to control external legacy equipment, signal external microcontrollers, or trigger external control logic implemented by other devices.

Continue to experiment with additional code to set the state of the GPIO based upon the state or outcome of some application code.  If you use a loop based upon some time condition, you can create a periodic or cyclical square wave.

<img src="images/ultra96_gpio_scope_periodic_cycle.png">

In [ ]:
import time
starttime=time.time()
while True:
  print("tick")
  time.sleep(1.0 - ((time.time() - starttime) % 1.0))
  ! echo 0 > /sys/class/gpio/gpio383/value
  print("tock")
  time.sleep(1.0 - ((time.time() - starttime) % 1.0))
  ! echo 1 > /sys/class/gpio/gpio383/value

Many thanks to the tips and tricks I got from Fred K. who steered me in the right direction!